In [235]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [294]:


class webscraper:

    def __init__(self):
        self.premier_league = { 'premier_league': []}
        self.team_stats = { 'team_stats': []}
        self.player_stats = { 'player_stats': []}

    def handle_blockers(self, driver):
        try:
            accept = driver.find_element(By.ID, "onetrust-accept-btn-handler")
            accept.click()
        except:
            pass

        try:
            accept = driver.find_element(By.ID, "advertClose")
            accept.click()
        except:
            pass
        
    def get_team_stats(self, driver):

        self.handle_blockers(driver)
        # print('here 1')
        time.sleep(5)
        # print('here 2')
        body = driver.find_element(By.CSS_SELECTOR, 'tbody[class="league-table__tbody isPL"]')
        # print('here 3')
        for position in range(1,21):
            # print('here 4')
            tr = body.find_element(By.CSS_SELECTOR, f'tr[data-position="{position}"]')
            # print('here 5')
            td = tr.find_element(By.CSS_SELECTOR, 'td[class="league-table__team team"]')
            # print('here 6')
            a = td.find_element(By.TAG_NAME, 'a')
            # print('here 7')
            # print('here 8')
            a.click()

            time.sleep(2)

            nav = driver.find_element(By.CSS_SELECTOR, 'nav[class="club-navigation wrapper"]')
            ul = nav.find_element(By.CSS_SELECTOR, 'ul[class="tablist club-navigation__nav"]')
            li = ul.find_elements(By.TAG_NAME, 'li')

            for item in li:
                a = item.find_element(By.TAG_NAME, 'a')

                if a.get_attribute('data-text') == 'Stats':
                    a.click()
                    break
                    

            time.sleep(5)
            driver.execute_script("window.history.go(-2)")
            time.sleep(1)
    
    def get_premier_league(self, driver):

        self.handle_blockers(driver)
                
        time.sleep(5)
        

        dropdown = driver.find_element(By.CSS_SELECTOR, 'div[data-dropdown-block="compSeasons"]')
        dropdown.click()
        time.sleep(1)

        seasons_div = dropdown.find_element(By.CSS_SELECTOR, 'div[class="dropdownListContainer"]')
        season_ul = seasons_div.find_element(By.CSS_SELECTOR, 'ul[class="dropdownList"]')
        seasons_li = season_ul.find_elements(By.TAG_NAME, 'li')


        count_for_season = 1
        for season_li in seasons_li:
            if count_for_season > 1:
                dropdown = driver.find_element(By.CSS_SELECTOR, 'div[data-dropdown-block="compSeasons"]')
                dropdown.click()
                time.sleep(1)
            if (season_li.get_attribute('data-option-name') != 'All Seasons'):
                season_li.click()
                time.sleep(10)
                season = season_li.get_attribute('data-option-name')

                body = driver.find_element(By.CSS_SELECTOR, 'tbody[class="league-table__tbody isPL"]')

                for position in range(1,21):
                    tr = body.find_element(By.CSS_SELECTOR, f'tr[data-position="{position}"]')

                    team_name = tr.get_attribute('data-filtered-table-row-name')

                    td = tr.find_elements(By.TAG_NAME, 'td')

                    count = 1
                    for value in td:
                        innerHTML = value.get_attribute('innerHTML')
                        if innerHTML.lstrip('-').isdigit():
                            if count == 1:
                                played = innerHTML
                            elif count == 2:
                                won = innerHTML
                            elif count == 3:
                                drawn = innerHTML
                            elif count == 4:
                                lost = innerHTML
                            elif count == 5:
                                gf = innerHTML
                            elif count == 6:
                                ga = innerHTML
                            elif count == 7:
                                gd = innerHTML
                            elif count == 8:
                                points = innerHTML
                            else:
                                break

                            count += 1

                    self.premier_league['premier_league'].append(
                        {
                                'season': season,
                                'team_name': team_name,
                                'position': position,
                                'played': played,
                                'won': won,
                                'drawn': drawn,
                                'lost': lost,
                                'gf': gf,
                                'ga': ga,
                                'gd': gd,
                                'points': points,
                        }
                    )


                count_for_season += 1
            else:
                count_for_season = count_for_season

    def scrape_website(self, url):

        # Initialize Selenium webdriver
        driver = webdriver.Chrome()
        
        try:
            # Load the webpage
            driver.get(url)
            
            try:
                self.get_premier_league(driver)
            except Exception as e:
                print("An error occurred get_premier_league:", str(e))
                pass

            # try:
            #     self.get_team_stats(driver)
            # except Exception as e:
            #     print("An error occurred get_team_stats:", str(e))
            #     pass

            return self.premier_league
            
        
        except Exception as e:
            print("An error occurred:", str(e))
            print(Exception.__name__)
            # return None
        
        finally:
            # Close the browser window
            driver.quit()

In [295]:
url = 'https://www.premierleague.com/tables'
webscraper = webscraper()
premier_league = webscraper.scrape_website(url)

In [297]:
df = pd.DataFrame(premier_league['premier_league'])

In [299]:
df.to_excel('data/premier_league_data.xlsx', index=False)